In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE271700"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE271700"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE271700.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE271700.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE271700.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome Changes and Metabolic Outcomes After Bariatric Surgery in Adults With Obesity and Type 2 Diabetes"
!Series_summary	"We aimed to characterize bariatric surgery-induced transcriptome changes associated with diabetes remission and the predictive role of the baseline transcriptome."
!Series_overall_design	"We performed a whole-genome microarray in peripheral mononuclear cells at baseline (before surgery) and 2 and 12 months after bariatric surgery in a prospective cohort of 26 adults with obesity and type 2 diabetes. We applied machine learning to the baseline transcriptome to identify genes that predict metabolic outcomes."
Sample Characteristics Dictionary:
{0: ['gender: Female', 'gender: Male'], 1: ['age: 51', 'age: 43', 'age: 46', 'age: 41', 'age: 29', 'age: 33', 'age: 36', 'age: 44', 'age: 48', 'age: 40', 'age: 49', 'age: 50', 'age: 35', 'age: 47', 'age: 31', 'age: 28', 'age: 37', 'age: 39'], 2: ['type of surgery: Biliopacreatic Div

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Assess gene expression data availability
is_gene_available = True  # The background info mentions whole-genome microarray on PBMCs, so gene expression data should be available

# 2. Variable availability and data type conversion
# 2.1 Data Availability
trait_row = 3  # "phenotype: Responder" or "phenotype: Non-Responder" can be used to infer obesity response status
age_row = 1    # Age information is available
gender_row = 0  # Gender information is available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert phenotype response status to binary values."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    value = value.strip().lower()
    if "responder" not in value:
        return None
    
    if "non-responder" in value:
        return 0  # Non-responder
    elif "responder" in value:
        return 1  # Responder
    else:
        return None

def convert_age(value):
    """Convert age to continuous values."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the age value after the colon
    parts = value.split(":")
    if len(parts) < 2:
        return None
    
    try:
        age = float(parts[1].strip())
        return age
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary values (Female=0, Male=1)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    value = value.strip().lower()
    if "gender" not in value:
        return None
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Conduct initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The sample characteristics dictionary is already provided in the previous step output
    sample_char_dict = {0: ['gender: Female', 'gender: Male'], 
                        1: ['age: 51', 'age: 43', 'age: 46', 'age: 41', 'age: 29', 'age: 33', 'age: 36', 'age: 44', 
                            'age: 48', 'age: 40', 'age: 49', 'age: 50', 'age: 35', 'age: 47', 'age: 31', 'age: 28', 
                            'age: 37', 'age: 39'], 
                        2: ['type of surgery: Biliopacreatic Diversion without duodenal switch', 
                            'type of surgery: laparascopic mini gastric bypass', 
                            'type of surgery: Roux en Y gastric Bypass', 
                            'type of surgery: Gastric Resection', 
                            'type of surgery: Sleeve Gastrectomy'], 
                        3: ['phenotype: Responder', 'phenotype: Non-Responder'], 
                        4: ['tissue: PBMCs']}
    
    # Convert the dictionary to a DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:", preview)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features: {0: [1.0, 51.0, 0.0], 1: [0.0, 43.0, 1.0], 2: [nan, 46.0, nan], 3: [nan, 41.0, nan], 4: [nan, 29.0, nan], 5: [nan, 33.0, nan], 6: [nan, 36.0, nan], 7: [nan, 44.0, nan], 8: [nan, 48.0, nan], 9: [nan, 40.0, nan], 10: [nan, 49.0, nan], 11: [nan, 50.0, nan], 12: [nan, 35.0, nan], 13: [nan, 47.0, nan], 14: [nan, 31.0, nan], 15: [nan, 28.0, nan], 16: [nan, 37.0, nan], 17: [nan, 39.0, nan]}
Clinical data saved to ../../output/preprocess/Obesity/clinical_data/GSE271700.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '100038246_at',
       '10003_at', '100048912_at', '100049716_at', '10004_at', '10005_at',
       '10006_at', '10007_at', '10008_at', '100093630_at', '100093698_at',
       '10009_at', '1000_at', '100101467_at', '100101938_at', '10010_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers
# The identifiers shown are in the format 'number_at', which is typical of probe IDs
# from microarray platforms (like Affymetrix) rather than standard human gene symbols.
# Standard human gene symbols would typically be things like BRCA1, TP53, etc.
# These appear to be probe IDs that need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '503538_at', '29974_at', '2_at', '144571_at'], 'SPOT_ID': ['1', '503538', '29974', '2', '144571']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. First, check the columns in the gene annotation dataframe to find gene symbols
print("All columns in gene annotation dataframe:")
print(gene_annotation.columns.tolist())

# 2. Further examine the annotation data to understand its structure
print("\nMore rows from gene annotation dataframe:")
print(gene_annotation.head(10))

# The annotations seem to be missing proper gene symbols
# We need to check if we can find the gene symbols from the SOFT file directly
# First, let's look at some gene identifiers in our expression data
print("\nGene expression identifiers example:")
print(gene_data.index[:5])

# Let's check for SOFT file content that might contain gene-probe mappings
# We'll extract lines containing gene annotations
soft_content = None
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Search for platform annotation sections that might contain gene symbols
print("\nSearching for gene symbol information in SOFT file...")
platform_section = re.search(r'!Platform_table_begin.*?!Platform_table_end', soft_content, re.DOTALL)

if platform_section:
    print("Found platform annotation table in SOFT file.")
    # Extract a sample to see column structure
    lines = platform_section.group(0).split('\n')[:10]
    print("Sample of platform annotation table:")
    for line in lines:
        print(line)

# Since we can't find proper gene symbols in the annotation data,
# we'll use the probe IDs directly and rely on normalization of gene symbols
# in a later step or external mapping
print("\nUsing probe IDs directly due to lack of gene symbol information...")

# Create a simple mapping using probe IDs
# The IDs in gene_data have a format like '100009676_at'
# Let's extract just the numeric part before '_at' as our 'gene ID'
mapping_df = pd.DataFrame({'ID': gene_data.index})
mapping_df['Gene'] = mapping_df['ID'].str.replace('_at', '', regex=False)

print("\nSimple mapping dataframe preview:")
print(preview_df(mapping_df))

# Convert probe-level data to gene-level data using the mapping
# This will essentially pass through the probe data, but prepare it for later normalization
gene_data_mapped = gene_data.copy()

# Normalize and save the gene data
# This preserves the probe-level data but will allow downstream processes to handle it
gene_data_mapped.to_csv(out_gene_data_file)
print(f"\nGene data saved to {out_gene_data_file}")

# Note for future steps:
print("\nNOTE: The dataset requires external mapping for gene symbols.")
print("The current approach preserves probe IDs for later processing or external mapping.")


All columns in gene annotation dataframe:
['ID', 'SPOT_ID']

More rows from gene annotation dataframe:
             ID    SPOT_ID
0          1_at          1
1     503538_at     503538
2      29974_at      29974
3          2_at          2
4     144571_at     144571
5     144568_at     144568
6      53947_at      53947
7      51146_at      51146
8  100506677_at  100506677
9       8086_at       8086

Gene expression identifiers example:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '100038246_at'], dtype='object', name='ID')

Searching for gene symbol information in SOFT file...

Using probe IDs directly due to lack of gene symbol information...

Simple mapping dataframe preview:
{'ID': ['100009676_at', '10000_at', '10001_at', '10002_at', '100038246_at'], 'Gene': ['100009676', '10000', '10001', '10002', '100038246']}



Gene data saved to ../../output/preprocess/Obesity/gene_data/GSE271700.csv

NOTE: The dataset requires external mapping for gene symbols.
The current approach preserves probe IDs for later processing or external mapping.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data saved in Step 2
clinical_data_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data from {out_clinical_data_file}")
print("Clinical data columns:", clinical_data_df.index.tolist())
print("Clinical data preview:", clinical_data_df.head())

# The first row in the clinical data is our trait (response to surgery)
# Rename it from index 0 to the actual trait name for consistency
clinical_data_df = clinical_data_df.rename(index={0: trait})

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_df, normalized_gene_data)
print(f"Linked clinical and genetic data. Shape: {linked_data.shape}")
print("Linked data columns:", linked_data.columns.tolist())
print("Linked data preview sample:", linked_data.head(2))

# 3. Handle missing values systematically
# The trait column in our data is actually the first row from clinical data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Handled missing values. Remaining samples: {len(linked_data_cleaned)}")

# If we have no samples left, adjust strategy
if len(linked_data_cleaned) == 0:
    print("All samples were filtered out due to missing values. Using a different approach.")
    # Try to recover usable data - keep samples with valid trait values
    linked_data_cleaned = linked_data.dropna(subset=[trait])
    print(f"Samples with valid trait values: {len(linked_data_cleaned)}")
    
    # If we still have no data, dataset is not usable
    if len(linked_data_cleaned) == 0:
        is_biased = True
        note = "Dataset has no usable samples with valid trait values after missing value handling."
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=is_gene_available, 
            is_trait_available=True,
            is_biased=is_biased,
            df=linked_data,
            note=note
        )
        print("Dataset is not usable for trait-gene association studies.")
        exit()

# 4. Determine if trait and demographic features are biased
is_biased, cleaned_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)
print(f"Trait is biased: {is_biased}")

# 5. Conduct final quality validation
note = "Study examines bariatric surgery response in obesity patients. The trait represents response to bariatric surgery, not obesity itself."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True,
    is_biased=is_biased,
    df=cleaned_linked_data,
    note=note
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")

Normalized gene data saved to ../../output/preprocess/Obesity/gene_data/GSE271700.csv
Loaded clinical data from ../../output/preprocess/Obesity/clinical_data/GSE271700.csv
Clinical data columns: ['Obesity', 'Age', 'Gender']
Clinical data preview:             0     1     2     3     4     5     6     7     8     9    10  \
Obesity   1.0   0.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Age      51.0  43.0  46.0  41.0  29.0  33.0  36.0  44.0  48.0  40.0  49.0   
Gender    0.0   1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

           11    12    13    14    15    16    17  
Obesity   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
Age      50.0  35.0  47.0  31.0  28.0  37.0  39.0  
Gender    NaN   NaN   NaN   NaN   NaN   NaN   NaN  
Linked clinical and genetic data. Shape: (91, 3)
Linked data columns: ['Obesity', 'Age', 'Gender']
Linked data preview sample:    Obesity   Age  Gender
0      1.0  51.0     0.0
1      0.0  43.0     1.0
Handled missing values. Remaining 